In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [3]:
#df = pd.DataFrame(columns = ['Best Model ID','Holding Cost','Model Nodes', 'Test Prediction Cost' ])


In [3]:
import random
import torch
import numpy as np
import os
from copy import deepcopy
import datetime
from tqdm.notebook import tqdm, tnrange,trange
from torch.utils.tensorboard import SummaryWriter
import params

from train_utils import *

model_type = 'simple_fc'  # 'att_context'
id = 'simple_fc'

# Set hyperparams:
missing = False
num_epochs = params.NUM_EPOCHS
batch_size = params.BATCH_SIZE
anneal_factor = params.ANNEALING_FACTOR
patience = params.PATIENCE

seed = np.random.randint(0,100000) ## change seed 

torch.cuda.synchronize() # comment
torch.cuda.empty_cache()

# set random seeds
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

# Other imports now
from data_utils import SalesDataset
from torch.utils.data import DataLoader

TRAIN = 'train.npz'
TEST = 'test.npz'

train_data = SalesDataset(TRAIN, randomize = False)
test_data = SalesDataset(TEST, randomize = False)

train_loader = DataLoader(train_data, batch_size, shuffle = True)
test_loader = DataLoader(test_data, batch_size, shuffle = True)
val_loader = test_loader

#------------------#
# Model Definition #
#------------------#

from hyperband import random_nn_params
from model import DeepVendorSimple
from loss_function import EuclideanLoss, CostFunction

best_overall_model = None
best_overall_cost = 100000.
best_model_idx = 0


for model_idx in tnrange(params.NUM_MODELS): 

    hidden_layers, n_nodes, lr, weight_decay = random_nn_params()

    model = DeepVendorSimple(n_hidden = hidden_layers, n_nodes = n_nodes)

    torch.cuda.empty_cache()
    #torch.cuda.memory_summary()
    #device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    device = torch.device('cpu')
    print(device)
    model = model.to(device)

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay)

    criterion = EuclideanLoss(params.SHORTAGE_COST, params.HOLDING_COST)
    test_criterion = CostFunction(params.SHORTAGE_COST, params.HOLDING_COST)

    writer_path = os.path.join(id+'_idx_'+str(model_idx))
    writer = SummaryWriter(writer_path)

    best_model = None
    best_val_loss = 100000.0

    try:
        early_stopping = EarlyStopping(patience=patience, verbose=False)

        for epoch in tnrange(num_epochs, total = None): ### (range):
            torch.cuda.empty_cache()

            # Train model
            loss = discriminative_trainer(
                model=model,
                data_loader=train_loader,
                optimizer=optimizer,
                criterion=criterion)
            #print(f'epoch: {epoch}, loss: {loss}')

            # log in tensorboard
            writer.add_scalar('Training/Prediction Loss', loss, epoch)

            # Eval model
            loss = discriminative_evaluate(model, val_loader, test_criterion)
            writer.add_scalar('Validation/Prediction Cost', loss, epoch)

            if loss < best_val_loss:
                best_val_loss = loss
                best_model = deepcopy(model)

            # Anneal LR
            early_stopping(loss, model)
            if early_stopping.early_stop:  # and epoch > params.MIN_EPOCH:
                #print("Early stopping")
                break

    except KeyboardInterrupt:
        print('Stopping training. Now testing')

    # Test the model
    model = best_model
    torch.cuda.empty_cache()
    loss= discriminative_evaluate(model, test_loader, test_criterion)
    print('Test Prediction Cost: ', loss)

    if loss < best_overall_cost:
        best_overall_model = deepcopy(model)
        best_overall_cost = loss
        best_model_idx = model_idx
        best_model_nodes = n_nodes
        
torch.save(model.state_dict(), os.path.join('best_model.pth'))


Generating network with 4 layers, each with [12 19 10  1] nodes
(2, array([12, 19, 10,  1]), 0.0009465733601508512, 0.0005657331373726005)


Generating network with 4 layers, each with [12 13  8  1] nodes
cpu


  5%|████▎                                                                          | 68/1250 [00:00<00:01, 668.23it/s]

EarlyStopping counter: 1 out of 10


 11%|████████▎                                                                     | 134/1250 [00:00<00:01, 652.23it/s]

...

EarlyStopping counter: 1 out of 10


  5%|████▏                                                                          | 66/1250 [00:00<00:01, 654.96it/s]

EarlyStopping counter: 1 out of 10


100%|██████████████████████████████████████████████████████████████████████████████| 313/313 [00:00<00:00, 1684.92it/s]


Test Prediction Cost:  2484.9590834352534



In [ ]:
df = pd.read_csv('LogTable.csv',index_col= False)
#df=df.iloc[: , 1:]

In [4]:
data = {'Best Model ID' : best_model_idx,'Holding Cost': params.HOLDING_COST,
        'Model Nodes' : list(best_model_nodes),'Test Prediction Cost' : best_overall_cost}

In [ ]:
df = df.append(data, ignore_index= True)

In [7]:
df.to_csv('LogTable.csv')

## GLPK Linear Program

In [42]:
A = pd.read_csv('train_Ice.csv', index_col= False)
A = A.iloc[:251 , 1:]

In [45]:
## Dataframe in .dat form for glpk
A.head()

,M,T,W,R,F,Tmax,Tmin,FeelsLike,Precipitation,WindSpeed,CloudCover,RelativeHumidity,Demand
0,0,1,0,0,0,69.3,52.2,59.4,0.00,6.6,66.6,79.17,290
1,0,0,1,0,0,64.5,26.9,44.8,0.00,11.5,39.7,44.39,922
2,1,0,0,0,0,69.2,37.2,35.3,0.00,8.7,24.9,41.43,858
3,0,0,0,0,1,50.7,36.4,32.3,0.04,15.9,64.5,78.16,872
4,1,0,0,0,0,68.7,50.9,58.5,0.00,4.2,69.3,79.99,996


In [41]:
A.to_csv('train_Ice.csv')

In [78]:
def ToGLPKdat(Df):
    """Df should be a DataFrame
    
    Quick and Dirty dat writer for Matrix Game 
    GLPK model. Df is Payoff Matrix."""
    Lines = 'set ROWS := '
    for row in Df.index:
        Lines += '%s ' % row
    Lines += ';\nset COLS := '
    for col in Df.columns[0:-1]:
        Lines += '%s ' % col
    Lines +=';\n\nparam B: '
    for col in Df.columns[-1]:
        Lines += '%s' % col
    Lines += ':= '
    Lines += '\n%s ' % Df['Demand']
    Lines += '; \n\nparam A: '
    for col in Df.columns[0:-1]:
        Lines += '%s ' % col
    Lines += ':='
    for row in Df.index:
        Lines += '\n%s ' % row 
        for col in Df.columns[0:-1]:
            Lines += '%s ' % Df.loc[row,col] 
    Lines += '; \n \nend;'
    with open('Training_Ice.dat','w') as FileObject:
        print('Writing to Training_Ice.dat')
        FileObject.write(Lines)
    return Lines

In [79]:
print(ToGLPKdat(A))

Writing to Training_Ice.dat
set ROWS := 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 ;
set COLS := M T W R F Tmax Tmin FeelsLike Precipitation WindSpee

In [81]:
%%script glpsol -m /dev/stdin -d Training_Ice.dat

set ROWS;
set COLS;

param  A{i in ROWS,j in COLS};
param B{i in ROWS};
param c_u :=0.5;
param c_o :=1.0;
#param Beta >= 0;

#param n :=;

var s{j in COLS} >=0;
var t{j in COLS} >=0;
var Beta{COLS} >=0;

# Objective function
minimize ExpectedCost:
sum{j in COLS}(c_u*s[j] + c_o*t[j]);
    
    
# Constraints    
subject to Underage{i in ROWS,j in COLS}:
s[j] >= B[i] - A[i,j]*Beta[j];

subject to Overage{i in ROWS,j in COLS}:
t[j] >=  A[i,j]*Beta[j] - B[i]  ;

subject to S{j in COLS}:
s[j] >= 0;

subject to T{j in COLS}:
t[j] >= 0;

OrderQuantity = 
solve;

display Beta;
display ExpectedCost;
 

end;

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 -m /dev/stdin -d Training_Ice.dat
Reading model section from /dev/stdin...
41 lines were read
Reading data section from Training_Ice.dat...
Training_Ice.dat:510: warning: final NL missing before end of file
510 lines were read
Generating ExpectedCost...
Generating Underage...
Generating Overage...
Generating S...
Generating T...
Model has been successfully generated
GLPK Simplex Optimizer, v4.65
6049 rows, 36 columns, 9806 non-zeros
Preprocessing...
3156 rows, 34 columns, 6312 non-zeros
Scaling...
 A: min|aij| =  1.000e-02  max|aij| =  1.000e+02  ratio =  1.000e+04
GM: min|aij| =  2.340e-01  max|aij| =  4.273e+00  ratio =  1.826e+01
EQ: min|aij| =  5.477e-02  max|aij| =  1.000e+00  ratio =  1.826e+01
Constructing initial basis...
Size of triangular part is 3156
      0: obj =   3.797000000e+03 inf =   1.260e+05 (1259)
   1259: obj =   6.129246656e+03 inf =   0.000e+00 (0) 12
*  1261: obj =   6.125974040e+03 